In [0]:
%sql
select distinct * from dev.siddi_bronze.sales where order_id is not null;

In [0]:
df=spark.read.table("dev.siddi_bronze.sales")

In [0]:
df1=df.dropDuplicates().dropna().drop("ingestion_date")
df1.write.mode("overwrite").saveAsTable("dev.siddi_silver.sales_cleaned")


In [0]:
%sql
select * from dev.siddi_bronze.products

In [0]:
%sql
create or replace table dev.siddi_silver.products_scd1 (product_id int , product_name string, price double, category string)

In [0]:
%sql
merge into dev.siddi_silver.products_scd1 as t
 using dev.siddi_bronze.products as s
  on t.product_id = s.product_id
   when matched and s.operation='DELETE'
then delete
when matched and s.operation='UPDATE' then update
   set
   t.product_name=s.product_name,t.price=s.product_price,t.category=s.product_category
   when not matched then
   insert(product_id,product_name,price,category)
   values(s.product_id,s.product_name,s.product_price,s.product_category)


In [0]:
%sql
select * from dev.siddi_silver.products_scd1